# Analysis

This notebook contains the analysis of the data tracked on
[Weights & Biases](https://wandb.ai/).


## Setup

---

We will first setup everything so that we can easily analyse the experiment
results. This includes importing the necessary libraries, setting paths, loading
the experiment results from W&B.


In [ ]:
# ruff: noqa
%load_ext autoreload
%autoreload 2

# Bult-in modules
import os
import sys

sys.path.insert(0, "..")

# Ignore warnings
import warnings

warnings.filterwarnings("ignore")

# External modules
# - Data Representation
import pandas as pd
import numpy as np

# - Data Visualization
from matplotlib import pyplot as plt
import seaborn as sns
import plotnine as pn

# - Machine Learning
import torch

# - Experiment Configuration and Logging
import wandb

# Custom modules
from utils import eval_utils as utils

In [ ]:
# Setup of global variables
ROOT_DIR = os.path.dirname(os.path.abspath("."))
ARTIFACT_DIR = os.path.join(ROOT_DIR, "artifacts")
FIGURE_DIR = os.path.join(ROOT_DIR, "report", "figures")
TABLE_DIR = os.path.join(ROOT_DIR, "report", "tables")

os.makedirs(FIGURE_DIR, exist_ok=True)
os.makedirs(TABLE_DIR, exist_ok=True)

METHODS = ["baseline", "baseline_pp", "matchingnet", "protonet", "maml"]
METHODS_WITH_SOT = []
for method in METHODS:
    METHODS_WITH_SOT.append(method)
    METHODS_WITH_SOT.append(method + "_sot")

STYLED_METHODS = ["Baseline", "Baseline++", "MatchingNet", "ProtoNet", "MAML"]
STYLED_METHODS_WITH_SOT = []
for method in STYLED_METHODS:
    STYLED_METHODS_WITH_SOT.append(method)
    STYLED_METHODS_WITH_SOT.append(method + " (SOT)")

styled_methods_dict = dict(zip(METHODS, STYLED_METHODS))


def get_name(name, sot=False):
    return styled_methods_dict[name] + (" (SOT)" if sot else "")

In [ ]:
# Settings
REPORT = True
sns.set_style("dark")
sns.set_palette("YlGn_r" if REPORT else "RdBu")

In [ ]:
# Initialize wandb
WANDB_PROJECT = "few-shot-benchmark"
WANDB_ENTITY = "metameta-learners"

# Initialize W&B API
api = wandb.Api()

# Get all runs
runs = api.runs(f"{WANDB_ENTITY}/{WANDB_PROJECT}")

## Experiment 1: Benchmark

---

All models on all datasets with and without SOT for fixed few-shot learning
setting (5-way 5-shot).


### Loading Experiment Data


In [ ]:
# Get all runs for experiment `benchmark`
GROUP = "new-tuned-benchmark"

exp1_runs = [run for run in runs if run.group == GROUP and run.state == "finished"]
print(f"✅ Found {len(exp1_runs)} runs")

Next, we'll load all runs from the given experiment group into a single
dataframe.


In [ ]:
df_exp1 = utils.load_to_df(exp1_runs)
print(f"✅ Loaded {len(df_exp1)} runs.")

df_exp1.head()

We check the number of runs for each unique experiment to verify that we have
the same number of runs for each experiment.


In [ ]:
# Number of runs per experiment
experiment_config = [("config", c) for c in df_exp1.config.columns]
df_exp1.groupby(experiment_config).size().reset_index(name="count")

### Grouping

Each experiment is uniquely identified by the following parameters:

- `dataset`: The dataset used (`swissprot`, `tabula_muris`)
- `method`: The model used (`baseline`, `baseline_pp`, `protonet`,
  `matchingnet`, `maml`)
- `use_sot`: Whether to include the SOT module (`True`, `False`)
- `n_way`: The number of classes in each episode
- `n_shot`: The number of support samples per class in each episode

For each experiment setting, there are multiple trained models because of
hyper-parameter tuning. We will group the runs by the above parameters and only
use the best-performing model on the validation set for the following analysis.


In [ ]:
# Group tuning runs by experiment configuration
df_exp1_best = utils.get_best_run(df_exp1, metric=("eval", "val/acc"))
print(f"✅ Filtered to {len(df_exp1_best)} best runs.")

# Let's also save two separate dataframes for the two different datasets
df_exp1_best_tm = df_exp1_best[df_exp1_best[("config", "dataset")] == "tabula_muris"]
df_exp1_best_sp = df_exp1_best[df_exp1_best[("config", "dataset")] == "swissprot"]

df_exp1_best.head()

### Train / Val / Test Performance for all models

Here, we plot a simple bar plot for all methods (5 methods, each with and
without SOT) on all three splits (train, val, test). Performances are shown in
two separate plots for Swissprot and Tabula Muris


In [ ]:
# Performance by split for all methods
fig, axs = plt.subplots(nrows=2, figsize=(20, 8))
fig.tight_layout(pad=3.0)


def pivot_acc(df):
    tmp = []
    for i, best_model in df.iterrows():
        for split in ["train", "val", "test"]:
            method_name = get_name(
                best_model[("config", "method")], best_model[("config", "use_sot")]
            )
            tmp.append(
                {
                    "method": method_name,
                    "split": split,
                    "acc": best_model[("eval", f"{split}/acc")],
                }
            )
    return pd.DataFrame(tmp)


sns.barplot(
    pivot_acc(df_exp1_best_tm),
    x="method",
    y="acc",
    hue="split",
    order=STYLED_METHODS_WITH_SOT,
    ax=axs[0],
)
sns.barplot(
    pivot_acc(df_exp1_best_sp),
    x="method",
    y="acc",
    hue="split",
    order=STYLED_METHODS_WITH_SOT,
    ax=axs[1],
)
# Set title
axs[0].set_title("Tabula Muris", fontweight="bold")
axs[1].set_title("SwissProt", fontweight="bold")

# Disable legend on first subplot
axs[0].get_legend().set_title("Split")
axs[1].get_legend().remove()

# Set y-axis limits
axs[0].set_ylim(0, 100)
axs[1].set_ylim(0, 100)

for ax in axs:
    ax.set_xlabel("Method")
    ax.set_ylabel("Acc. (%)")

fig.savefig(os.path.join(FIGURE_DIR, "benchmark-split-perf.pdf"), bbox_inches="tight")
print(f"✅ Saved figure to {FIGURE_DIR}.")

### Performance by method with and without SOT

Here, we compare the performance of the different methods with and without SOT.
The left subplot shows the test performance on the Tabula Muris dataset, while
the right subplot shows the test performance on the Swissprot dataset.


In [ ]:
# Performance by method with and without SOT
fig, axs = plt.subplots(ncols=2, figsize=(20, 4))

sns.barplot(
    df_exp1_best_tm,
    x=("config", "method"),
    y=("eval", "test/acc"),
    hue=("config", "use_sot"),
    order=METHODS,
    ax=axs[0],
)

sns.barplot(
    df_exp1_best_sp,
    x=("config", "method"),
    y=("eval", "test/acc"),
    hue=("config", "use_sot"),
    ci="sd",
    order=METHODS,
    ax=axs[1],
)

# Set title
axs[0].set_title("Tabula Muris", fontweight="bold", fontsize=14)
axs[1].set_title("SwissProt", fontweight="bold", fontsize=14)

# Disable legend on first subplot
axs[0].get_legend().set_title("SOT")
axs[1].get_legend().set_title("SOT")

# Set axis labels
axs[0].set_xticklabels([get_name(name.get_text())
                       for name in axs[0].get_xticklabels()])
axs[1].set_xticklabels([get_name(name.get_text())
                       for name in axs[1].get_xticklabels()])

for ax in axs:
    ax.set_ylabel("Test Acc. (%)", fontsize=18)
    ax.set_xlabel("Method", fontsize=18)

# Remove legend in left plot
axs[0].get_legend().remove()

# Increasing tick size
axs[0].tick_params(labelsize=14)
axs[1].tick_params(labelsize=14)

axs[0].set_ylim(0, 105)
axs[1].set_ylim(0, 105)

# Adding padding between ticks and labels
axs[0].xaxis.labelpad = 10
axs[1].xaxis.labelpad = 10

# Add percentage to each bar
for p in axs[0].patches:
    axs[0].annotate(
        f"{p.get_height():.1f}",
        (p.get_x() + p.get_width() / 2.0, p.get_height()),
        ha="center",
        va="center",
        xytext=(0, 10),
        textcoords="offset points",
        fontsize=15,
    )

# Add percentage to each bar
for p in axs[1].patches:
    axs[1].annotate(
        f"{p.get_height():.1f}",
        (p.get_x() + p.get_width() / 2.0, p.get_height()),
        ha="center",
        va="center",
        xytext=(0, 10),
        textcoords="offset points",
        fontsize=15,
    )

# Add percentage to each bar
errors = df_exp1_best_sp[("eval", "test/acc_ci")]
df_exp1_best_sp["method_order"] = pd.Categorical(
    df_exp1_best_sp[("config", "method")], categories=METHODS, ordered=True
)
df_exp1_best_tm["method_order"] = pd.Categorical(
    df_exp1_best_tm[("config", "method")], categories=METHODS, ordered=True
)

# Sort by the custom order
sp_errors = df_exp1_best_sp.sort_values(["method_order", ("config", "use_sot")])[
    ("eval", "test/acc_ci")
]
tm_errors = df_exp1_best_tm.sort_values(["method_order", ("config", "use_sot")])[
    ("eval", "test/acc_ci")
]

for i, p in enumerate(axs[0].patches):
    axs[0].errorbar(
        p.get_x() + p.get_width() / 2.0,
        p.get_height(),
        yerr=sp_errors[i],
        fmt="none",
        color="black",
        capsize=5,
    )

for i, p in enumerate(axs[1].patches):
    axs[1].errorbar(
        p.get_x() + p.get_width() / 2.0,
        p.get_height(),
        yerr=tm_errors[i],
        fmt="none",
        color="black",
        capsize=5,
    )

# Save figure
name = "benchmark-method-perf" if REPORT else "present-benchmark-method-perf.png"
fig.savefig(os.path.join(FIGURE_DIR, name), bbox_inches="tight")
print(f"✅ Saved figure to {FIGURE_DIR}.")

In [ ]:
# df_best_runs_sp["method_order"] = pd.Categorical(
#     df_best_runs_sp[("config", "method")], categories=METHODS, ordered=True
# )
# 
# # Sort by the custom order
# df_best_runs_sp.sort_values(["method_order", ("config", "use_sot")])

### Hyperparameter Ablation

For each method including the SOT module we are searching over the following
hyperparameters:

- `lr`: The learning rate of the optimizer (`0.001`, `0.01`, `0.1`)
- `hidden_dim`: The hidden dimension of the backbone (`64`, `512`, `1024`)
- `sot_reg`: Regularization strength for the SOT module (`0.01`, `0.1`, `1.0`)
- `sot_dist`: The distance function used for the SOT module (`euclidean`,
  `cosine`)

This leads to a total of $3 \times 3 \times 3 \times 2 = 54$ different tuning
runs for each model. Let's first verify that we have all runs necessary and then
analyse the effect of the hyperparameters on the performance.


In [ ]:
# Show the best hyper-parameters for each method
df_exp1_best[["config", "hparams"]]

We can see the following:

- Using `cosine` distance seems to be better across all methods and datasets.
- A regularisation parameter during the SOT phase of `0.1` is preferred across
  almost all methods and datasets.
- A low learning rate is generally preferred `0.001`
- Feature dimensions vary across methods and datasets.


Next, we visualise the effect of each hyperparameter individually for both
datasets separately. We are averaging the results of all tuning runs for each
hyperparameter value and method.


In [ ]:
num_hparams = len(df_exp1.hparams.columns)

# Hyperparameter Effect on SwissProt for all methods
fig, axs = plt.subplots(ncols=num_hparams, figsize=(20, 4))
fig.suptitle("Effect of Hyperparameters on SwissProt", fontsize=16)
for ax, hparam in zip(axs, df_exp1.hparams.columns):
    utils.visualise_hparams(df_exp1, dataset="swissprot", hparam=hparam, ax=ax)

fig.savefig(os.path.join(FIGURE_DIR, "hparams-swissprot.pdf"),
            bbox_inches="tight")
print(f"✅ Saved figure to {FIGURE_DIR}.")

# Hyperparameter Effect on Tabula Muris for all methods
fig, axs = plt.subplots(ncols=num_hparams, figsize=(20, 4))
fig.suptitle("Effect of Hyperparameters on Tabula Muris", fontsize=16)
for ax, hparam in zip(axs, df_exp1.hparams.columns):
    utils.visualise_hparams(
        df_exp1, dataset="tabula_muris", hparam=hparam, ax=ax)

fig.savefig(os.path.join(FIGURE_DIR, "hparams-tabula-muris.pdf"),
            bbox_inches="tight")
print(f"✅ Saved figure to {FIGURE_DIR}.")

Next, we show the interaction of all hyper-parameters pairs and the method and
dataset performance.


In [ ]:
# Choose hyperparameters to visualise
param_cols = df_exp1[("hparams")].columns.tolist()
param_tuples = [("hparams", param) for param in param_cols]
param_tuples.append(("config", "method"))
# param_tuples.append(("config", "dataset"))
param_tuples
subset = df_exp1[
    df_exp1[("config", "use_sot")] 
    & df_exp1[("config","dataset")].isin(["swissprot"]) 
    # & (df_exp1[("eval", "test/acc")] >= 25.0)
]

rename_dict = {
    "lr": "LR",
    "feat_dim": "Backbone\nDimension",
    "sot_dist_metric": "SOT\nDistance\nMetric",
    "use_sot": "SOT",
    "sot_reg": "SOT\nReg.",
    "method": "Method",
    "dataset": "Dataset",
}

fig = utils.grid(
    subset, param_tuples, metric="mean", cmap="YlGn", figsize=(10, 10), rename_dict=rename_dict
)  # vmin, vmax automatically set

fig.savefig(
    os.path.join(FIGURE_DIR, "hparams-interaction-swissprot.png"),
    bbox_inches="tight",
    dpi=300,
)
print(f"✅ Saved figure to {FIGURE_DIR}.")

In [ ]:
# Choose hyperparameters to visualise
param_cols = df_exp1[("hparams")].columns.tolist()
param_tuples = [("hparams", param) for param in param_cols]
param_tuples.append(("config", "method"))
# param_tuples.append(("config", "dataset"))
param_tuples
subset = df_exp1[
    df_exp1[("config", "use_sot")]
    & df_exp1[("config", "dataset")].isin(["tabula_muris"])
    & (df_exp1[("eval", "test/acc")] >= 25.0)
]
fig = utils.grid(
    subset, param_tuples, metric="mean", cmap="YlGn", figsize=(10, 10)
)  # vmin, vmax automatically set

fig.savefig(
    os.path.join(FIGURE_DIR, "hparams-interaction-tabula_muris.png"),
    bbox_inches="tight",
    dpi=300,
)
print(f"✅ Saved figure to {FIGURE_DIR}.")

In [ ]:
# combined grid
param_cols = df_exp1[("hparams")].columns.tolist()
param_tuples = [("hparams", param) for param in param_cols]
param_tuples.append(("config", "method"))
# param_tuples.append(("config", "dataset"))

rename_dict = {
    "lr": "Learning\nRate",
    "feat_dim": "Backbone\nDimension",
    "sot_dist_metric": "SOT\nDistance\nMetric",
    "sot_reg": "SOT\nReg.",
    "method": "Method",
    "dataset": "Dataset",
    "baseline":"B",
    "baseline_pp":"B++",
    "matchingnet":"MN",
    "protonet":"PN",
    "maml":"MAML",
    "euclidean":"EUCL",
    "cosine":"COS",
}

subset1 = df_exp1[
    df_exp1[("config", "use_sot")]
    & df_exp1[("config", "dataset")].isin(["tabula_muris"])
    & (df_exp1[("eval", "test/acc")] >= 25.0)
    ]

subset2 = df_exp1[
    df_exp1[("config","use_sot")]
    & df_exp1[("config","dataset")].isin(["swissprot"])
    & (df_exp1[("eval","test/acc")] >= 25.0)
]

fig = utils.combined_grid(
    subset1, subset2, param_tuples, metric="mean", cmap="YlGn", figsize=(10, 10), rename_dict=rename_dict
)  # vmin, vmax automatically set

fig.savefig(
    os.path.join(FIGURE_DIR, "hparams-interaction-combined.png"),
    bbox_inches="tight",
    dpi=300,
)
print(f"✅ Saved figure to {FIGURE_DIR}.")

### Generate LaTeX table


In [ ]:
# Parse the results for both the report
tm_res = utils.exp2results(df_exp1_best_tm)
sp_res = utils.exp2results(df_exp1_best_sp)

# Create a MultiIndex for the results where the first level is the dataset
tm_res = tm_res.set_index("Method")
tm_res.index = pd.MultiIndex.from_product([["TM"], tm_res.index])
sp_res = sp_res.set_index("Method")
sp_res.index = pd.MultiIndex.from_product([["SP"], sp_res.index])

# Concatenate the results
df_results = pd.concat([tm_res, sp_res])

# Get the latex
latex = utils.exp2latex(df_results)

# Save the latex
with open(os.path.join(TABLE_DIR, "tuned-benchmark.tex"), "w") as f:
    f.write(latex)

df_results

## Experiment 2: Way-Shot Analysis

---

Varying the number of shots per class.


In [ ]:
# Load experiment data for `way-shot` experiment
GROUP = "new-way-shot"

# Filter runs by group
group_runs = [run for run in runs if run.group ==
              GROUP and run.state == "finished"]
print(f"✅ Found {len(group_runs)} runs")

In [ ]:
# Load runs into dataframe
df_runs = utils.load_to_df(group_runs)
print(f"✅ Loaded {len(df_runs)} runs.")

df_runs.head()

Only keep the best run for each experiment configuration. This only has an
effect if hyperparameter tuning was performed.


In [ ]:
# Group tuning runs by experiment configuration
df_best_runs = utils.get_best_run(df_runs, metric=("eval", "val/acc"))
print(f"✅ Filtered to {len(df_best_runs)} best runs.")

df_best_runs.head()

### Shot-Way-Analysis

Display the test/acc as a function of the number of shots per class and the
number of classes to distinguish between the different methods for ProtoNet
without and with SOT.


In [ ]:
# Set context to paper
sns.set_context("paper")
sns.set_style("dark")

# Plot test/acc vs. n_shot for SOT and non-SOT methods
fig, axs = plt.subplots(ncols=2, figsize=(7, 3.5), sharey=True)

# test/acc ~ n_shot
sns.scatterplot(
    data=df_best_runs,
    x=("config", "n_way"),
    y=("eval", "test/acc"),
    hue=("config", "use_sot"),
    alpha=0.25,
    legend=True,
    ax=axs[0],
)
sns.lineplot(
    data=df_best_runs,
    x=("config", "n_way"),
    y=("eval", "test/acc"),
    hue=("config", "use_sot"),
    legend=False,
    ax=axs[0],
)

# test/acc ~ n_way
sns.scatterplot(
    data=df_best_runs,
    x=("config", "n_shot"),
    y=("eval", "test/acc"),
    hue=("config", "use_sot"),
    alpha=0.25,
    legend=False,
    ax=axs[1],
)
sns.lineplot(
    data=df_best_runs,
    x=("config", "n_shot"),
    y=("eval", "test/acc"),
    hue=("config", "use_sot"),
    legend=False,
    ax=axs[1],
)

# Set label size
labsize = 14

# Set title of the legend, adjust font size
axs[0].get_legend().set_title("SOT", prop={"size": 1})
for t in axs[0].get_legend().texts:
    t.set_fontsize(12)

# Set axis labels
axs[0].set_xlabel("N-Way", size=labsize)
axs[1].set_xlabel("N-Shot", size=labsize)

# Add padding between label of x-axis and tick labels
axs[0].xaxis.labelpad = 10
axs[1].xaxis.labelpad = 10

# Set axis labels
axs[0].set_ylabel("Test. Acc. (%)", size=labsize)
axs[1].set_ylabel("")

# Make the ticks descroptors bigger
axs[0].tick_params(labelsize=12)
axs[1].tick_params(labelsize=12)

# Set the tight layout
fig.tight_layout(pad=1.0)

# Save figure
name = "way-shot" if REPORT else "present-way-shot.png"
fig.savefig(os.path.join(FIGURE_DIR, name), bbox_inches="tight")
print(f"✅ Saved figure to {FIGURE_DIR}.")

In [ ]:
# Get the dataframe with n_way and n_shot, and the test accuracy
way_shot = df_best_runs[
    [
        ("config", "n_way"),
        ("config", "n_shot"),
        ("eval", "test/acc"),
        ("config", "use_sot"),
    ]
]

# Flatten the multi-index
way_shot.columns = way_shot.columns.get_level_values(1)

# Split the dataframe into SOT and non-SOT
way_shot_sot = way_shot[way_shot["use_sot"]].drop(columns=["use_sot"])
way_shot = way_shot[~way_shot["use_sot"]].drop(columns=["use_sot"])

# Group by n_way and n_shot
way_shot = way_shot.groupby(["n_way", "n_shot"]).mean().reset_index()
way_shot_sot = way_shot_sot.groupby(["n_way", "n_shot"]).mean().reset_index()

# Pivot the dataframe
way_shot = way_shot.pivot(index="n_way", columns="n_shot", values="test/acc")
way_shot_sot = way_shot_sot.pivot(
    index="n_way", columns="n_shot", values="test/acc")

# Compute the diff
way_shot_diff = way_shot_sot - way_shot

# Setup the figure
fig, axs = plt.subplots(ncols=2, figsize=(12, 5))


# Plot the heatmap
sns.heatmap(
    way_shot_sot,
    annot=True,
    cmap="YlGn",
    vmin=50,
    vmax=100,
    ax=axs[0],
)

# Plot the heatmap
sns.heatmap(
    way_shot_diff,
    annot=True,
    cmap="RdBu_r",
    ax=axs[1],
    center=0,
    cbar_kws={"label": "Accuracy Difference (%)"},
)
for ax in axs:
    ax.set_xlabel("N-Shot")
    ax.set_ylabel("N-Way")
axs[1].set_ylabel("")

fig.tight_layout(pad=1.0)
name = "way-shot-heatmap" if REPORT else "present-way-shot-heatmap.png"
fig.savefig(os.path.join(FIGURE_DIR, name), bbox_inches="tight")
print(f"✅ Saved figure to {FIGURE_DIR}.")

In [ ]:
# Don't run the rest of the notebook
raise ValueError("Don't run the rest of the notebook.")

## Experiment 3: Understanding model performance

---

The goal of this section will be to understand the behaviour of the SOT module.
We will try to understand the improvements by looking at:

- **Embeddings during forward-pass**. Visualise the embeddings of support and
  query samples during episodes with and without SOT enabled.
- **Visualise the self-optimal transport plan.** Visualise the self-optimal
  transport plan for a few episodes via a heat map.
- **Understand model prediction patterns and errors.** Visualise the model
  predictions and errors for a few episodes with and without SOT enabled.

To get started, we will load two pre-trained models from the benchmarking
experiment. We will use two instances of `protonet` that were both trained on
the `tabula_muris` dataset. The first model was trained with the default
configuration, while the second model was trained with the same configuration
but with the `use_sot` flag set to `True`.


In [ ]:
# Experiments
GROUP = "model-behaviour"

# Filter runs by group
runs = [run for run in runs if run.group == GROUP and run.state == "finished"]
print(f"✅ Loaded {len(runs)} runs")

# Load runs into dataframe
df_runs = utils.load_to_df(runs)
df_runs.head()

In [ ]:
# Initialise data loaders and model
models = []
for run in runs:
    # Load data loaders and model
    train_loader, val_loader, test_loader, model = utils.init_all(run)
    models.append(model)

print(f"✅ Initialised data loader and model.")

In [ ]:
# Download artifact (model weights)
for run in runs:
    if not os.path.exists(os.path.join(ARTIFACT_DIR, run.id)):
        utils.download_artifact(
            api,
            wandb_entity=WANDB_ENTITY,
            wandb_project=WANDB_PROJECT,
            artifact_dir=ARTIFACT_DIR,
            run_id=run.id,
        )
    print(f"✅ Downloaded artifact for run {run.id}.")

In [ ]:
# Load model weights
weight_path = os.path.join(ARTIFACT_DIR, runs[0].id, "best_model.pt")
models[0].load_state_dict(torch.load(weight_path))

weight_path = os.path.join(ARTIFACT_DIR, runs[1].id, "best_model.pt")
models[1].load_state_dict(torch.load(weight_path))

print(f"✅ Loaded all model weights.")

In [ ]:
# Evaluate performance
print("\nEvaluating X...")
models[0].test_loop(train_loader)
models[0].test_loop(val_loader)
models[0].test_loop(test_loader)

print("Evaluating Y...")
models[1].test_loop(train_loader)
models[1].test_loop(val_loader)
models[1].test_loop(test_loader)


print(f"✅ Evaluated both models.")

We have correctly loaded the model weights by confirming the performance on the
`train`, `val` and `test` split for ProtoNet on SwissProt with and without SOT.


### Visualise Embeddings


In [ ]:
# Visualise episode for Protnet w/ SOT
for loader in [train_loader, val_loader, test_loader]:
    utils.visualise_episode(loader, models[1])

In [ ]:
# Visualise episode for Protnet w/o SOT
for loader in [train_loader, val_loader, test_loader]:
    utils.visualise_episode(loader, models[2])

### Visualise SOT transport plan

Here we visualise the self-optimal transport plan for a few episodes.


In [ ]:
# Visualise transport plan of X
fig, axs = plt.subplots(ncols=3, figsize=(20, 5))
for ax, loader in zip(axs, [train_loader, val_loader, test_loader]):
    x, _ = next(iter(loader))
    utils.visualise_transport_plan(x, models[0], ax=ax)

In [ ]:
# Visualise transport plan for ProtoNet w/ SOT w/o SE on splits
fig, axs = plt.subplots(ncols=3, figsize=(20, 5))
for ax, loader in zip(axs, [train_loader, val_loader, test_loader]):
    x, _ = next(iter(loader))
    utils.visualise_transport_plan(x, models[1], ax=ax)

In [ ]:
# Visualise transport plan for Baseline w/ SOT on splits
fig, axs = plt.subplots(ncols=3, figsize=(20, 5))
for ax, loader in zip(axs, [train_loader, val_loader, test_loader]):
    x, _ = next(iter(loader))
    utils.visualise_transport_plan(x, models[2], ax=ax)

In [ ]:
# Compare transport plan of ProtoNet w/ and w/o SE and w/ SOT
fig, axs = plt.subplots(ncols=3, figsize=(16, 5))
fig.tight_layout(pad=3.0)

x, _ = next(iter(test_loader))

utils.visualise_transport_plan(x, models[0], ax=axs[0])
utils.visualise_transport_plan(x, models[1], ax=axs[2])

print()

In [ ]:
# Compare transport plan of ProtoNet with Baseline on test split
fig, axs = plt.subplots(ncols=2, figsize=(10, 5))
fig.tight_layout(pad=3.0)

x, _ = next(iter(test_loader))
utils.visualise_transport_plan(x, models[1], ax=axs[0])
utils.visualise_transport_plan(x, models[2], ax=axs[1])

# Remove heatbar from axs[0]
bar = axs[0].collections[0].colorbar
axs[0].collections[0].colorbar.remove()
axs[1].collections[0].colorbar.remove()
pn_acc = axs[0].title.get_text().split(" ")[-1]
b_acc = axs[1].title.get_text().split(" ")[-1]
axs[0].set_title(f"ProtoNet SOT Embeddings (Acc. = {pn_acc}", fontsize=14)
axs[1].set_title(f"Baseline SOT Embeddings (Acc. = {b_acc}", fontsize=14)

# Add colbar in between the two plots
# axs[0].figure.colorbar(bar, ax=axs, location="right")p
cbar_ax = fig.add_axes([0.485, 0.25, 0.02, 0.5])
fig.colorbar(axs[0].collections[0], cax=cbar_ax, orientation="vertical")

# Save figure
fig.savefig(os.path.join(FIGURE_DIR, "sot-embeddings.pdf"), bbox_inches="tight")
print(f"✅ Saved figure to {FIGURE_DIR}.")

### Visualise confusion patterns

p fig.subplots_adjust(bottom=0.2)


In [ ]:
# Confusion matrix for Protonet w/ SOT on all splits
fig, axs = plt.subplots(ncols=3, figsize=(20, 5))
for ax, loader in zip(axs, [train_loader, val_loader, test_loader]):
    utils.visualise_confusion_matrix(loader, models[0], ax=ax)

In [ ]:
# Confusion matrix for Protonet w/o SOT on all splits
fig, axs = plt.subplots(ncols=3, figsize=(20, 5))
for ax, loader in zip(axs, [train_loader, val_loader, test_loader]):
    utils.visualise_confusion_matrix(loader, models[1], ax=ax)